# FER2013 이미지 데이터 전처리 스크립트
이 스크립트는 FER2013 데이터셋의 이미지들을 전처리하는 작업을 자동화합니다.  
 주요 목적은 이미지들의 크기를 조정하고, 필요한 형식으로 저장하는 것입니다.

In [1]:
import os
from glob import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
CROP_SIZE = (224,224) #사진크기 지정

In [2]:
test_or_train = 'train'

In [3]:
class Datapreprocess:
    def __init__(self):
        # 클래스 초기화 시 사용되는 변수들을 설정합니다.
        self.img = None
        self.save_path = None
        self.file_name = None
        self.new_file_path = None
        self.curr_emotion = None
        self.curr_dir = os.getcwd()
        self.data_lst = []
        
    def _resize_img(self):
        # 이미지 크기를 변경하고, 그레이스케일로 변환하는 함수입니다.
        img = self.img.copy()
        img = cv2.resize(img, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self.img = img
        
    def _read_img(self,file_name):
        # 이미지를 읽어오는 함수입니다.
        self.img_path = os.path.join(self.curr_dir,'fer2013',test_or_train,self.curr_emotion,file_name)
        self.img = cv2.imread(self.img_path)
        
    def _save_img(self):
        # 이미지를 저장하는 함수입니다.
        self._create_path(os.path.join(self.curr_dir,'fer2013','resize',test_or_train,self.curr_emotion))
        cv2.imwrite(self.new_file_path,self.img)
        self.img = None

    def _record(self):
        # 데이터 정보를 기록하고 CSV 파일로 저장하는 함수입니다.
        self.data_lst.append({"path": self.new_file_path, "label": self.curr_emotion})
        df = pd.DataFrame(self.data_lst)
        self._create_path(os.path.join(self.curr_dir,'fer2013','resize',test_or_train,'csvs'))
        df.to_csv(f"./fer2013/resize/{test_or_train}/csvs/{self.curr_emotion}.csv")
        del df

    def _create_path(self,path):
        # 주어진 경로에 디렉토리가 없으면 생성하는 함수입니다.
        if not os.path.exists(path):
            os.makedirs(path)
            return path
        else:
            return path
            
    def work(self, file_name, emo):
        # 이미지 처리를 위한 메인 함수입니다.
        self.curr_emotion = emo
        self.file_name = file_name
        self._read_img(self.file_name)
        self._resize_img()
        self.new_file_path = f"./fer2013/resize/{test_or_train}/{self.curr_emotion}/{self.file_name}"
        self._save_img()
        self._record()

In [4]:
emotion = ['angry','embarrassed','happy','neutral','sad']
datapreprocess = Datapreprocess()

# 감정별로 이미지를 처리합니다.
for emo in emotion:
    datapreprocess.__init__()
    path = os.path.join(os.getcwd(),'fer2013',test_or_train,emo)
    files = os.listdir(path)
    for file in tqdm(files,desc = emo):
        datapreprocess.work(file,emo)

sad: 100%|████████████████████████████████████████████████████████████████████████| 4830/4830 [00:33<00:00, 145.45it/s]
